# SCP Toolbox Workshop

___A tutorial on generating dynamically feasible trajectories reliably and efficiently___

Monday, February 7, 2022

Rocky Mountain AAS GN&C Conference, Breckenridge, CO

In [1]:
import Pkg
Pkg.activate("..")

## these lines are required for local installations
# Pkg.develop(path="../../SCPToolbox.jl/")
# Pkg.precompile()

Pkg.add("JuMP")
Pkg.build("JuMP")

using SCPToolbox
using JuMP
using ECOS
using Ipopt
# using SCS
# using OSQP
using Printf

  Activating project at `~/GitHub/SCPToolbox_tutorial`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/GitHub/SCPToolbox_tutorial/Project.toml`
  No Changes to `~/GitHub/SCPToolbox_tutorial/Manifest.toml`


# Part 2: Simple Conic Linear Program Example

Solve the following nonconvex optimization problem as a simple conic linear program subproblem:

\begin{align}
\underset{z}{\text{minimize}} &\quad L(z) \triangleq z_2 \\
\text{subject to} &\quad \color{#000080}{z_2 - z_1^2 = 0}\\
 &\quad \color{#008000}{z_2 + 0.1 z_1 = 0.6}\\
 &\quad \color{#FF00FF}{z_1 \leq 0.85}
\end{align}

<center>
    <br />
    <img src="media/p2-crawling.png" width="450"/>
    <br />
    <b>Figure.</b>  A two-dimensional nonconvex toy problem that exemplifies a convex subproblem obtained during an SCP iteration.
    <br />
</center>

In this case, the cost function $L(z) = z_2$ and level curves of the cost are shown as gray dashed lines. The blue curve represents a nonconvex equality constraint, and its linearization is shown as the blue dash-dot line. Another convex equality constraint is shown in green, and a convex inequality constraint is shown as the vertical red dashed line. The trust region is the red circle centered at the linearization point $\bar{z}$, and has radius $\eta$. The optimal solution of the original (non-approximated) problem is shown as the black square. The convex subproblem is artificially infeasible. Without the trust region and the green constraint, it would also be artifically unbounded.  

To convert this problem into a CLP subproblem, we first generate (convex, linear) hyperplane
approximation of the nonconvex constraint. We also include virtual buffers $\nu$ to avoid artificial infeasibility, and a trust region upper bound $\eta$ to avoid artificial unboundedness.

Penalties for these terms are augmented to the cost function. Virtual control magnitude is penalized to incentivize convergence to zero once feasibility of the subproblem is obtained. In fact, the virtual control penalty has a higher weight in the augmented objective. Trust region radius is penalized to incentivize the optimal solution of the subproblem at the current iteration to stay close to the linearization point, where the approximation of the nonconvex constraint is valid. Furthermore, the trust region constraint precludes artificial unboundedness that may arise from the feasible set becoming unbounded due to the linearized constraint.

\begin{align}
\underset{z}{\text{minimize}} &\quad z_2 + w_{\text{tr}}\color{#FF0000}{\eta} + w_{\text{vb}}\color{#000080}{|\nu|} \\
\text{subject to} &\quad  \color{#000080}{(-2\bar{z}_1)z_1 + z_2 + \bar{z}_1^2 + \nu = 0}  \\
 &\quad \color{#008000}{z_2 + 0.1 z_1 = 0.6}\\
 &\quad \color{#FF00FF}{z_1 \leq 0.85} \\
 &\quad \color{#FF0000 }{\| z - \bar{z} \|_2 \leq \eta}
\end{align}

The subproblem described above is still not a CLP since the objective function is not linear. The virtual buffer penalty is converted to a conic constraint via the epigraph form by introducing a slack variable $\mu$.

\begin{align}
\underset{z}{\text{minimize}} &\quad z_2 + w_{\text{tr}}\color{#FF0000}{\eta} + w_{\text{vb}}\color{#000080}{\mu}\\
\text{subject to} &\quad  \color{#000080}{(-2\bar{z}_1)z_1 + z_2 + \bar{z}_1^2 + \nu = 0}  \\
 &\quad \color{#008000}{z_2 + 0.1 z_1 = 0.6}\\
 &\quad \color{#FF00FF}{z_1 \leq 0.85} \\
 &\quad \color{#FF0000 }{\| z - \bar{z} \|_2 \leq \eta} \\
 &\quad \color{#000080}{|\nu| \leq \mu}
\end{align}

## Constraints

In [2]:
# Linearized nonconvex constraint
lin_ncvx(z, ν, pbm) = @add_constraint(pbm, ZERO, (z, ν) -> (-2*cst["z̄"][1])*z[1] .+ z[2] .+ cst["z̄"][1]^2 .+ ν);

In [3]:
# Hyperplane constraint
hypln(z, pbm) = @add_constraint(pbm, ZERO, z -> z[2] + 0.1*z[1] - 0.6);

In [4]:
# Halfspace constraint
hlfsp(z, pbm) = @add_constraint(pbm, NONPOS, z -> z[1] - 0.85);

In [5]:
# Trust-region penalty constraint
tr_cnstr(z, η, pbm) = @add_constraint(pbm, SOC, (z, η) -> vcat(η, z .- cst["z̄"]));

In [6]:
# Virtual buffer penalty constraint
vb_cnstr(ν, μ, pbm) = @add_constraint(pbm, L1, (ν, μ) -> vcat(μ, ν));

## Cost function

In [7]:
cost_fun(η, μ, z, pbm) = @add_cost(pbm, (η, μ, z) -> z[2] .+ cst["wtr"]*η .+ cst["wvb"]*μ);

## SCP subproblem

In [8]:
function construct_subpbm(params)
    
    # Construct the subproblem
    pbm = ConicProgram(params;solver = params["solver"],solver_options = params["opts"])
    
    # Create variables
    z = @new_variable(pbm, 2, "z")
    ν = @new_variable(pbm, "ν")
    η = @new_variable(pbm, "η")
    μ = @new_variable(pbm, "μ")  
        
    # Linearized nonconvex constraint
    lin_ncvx(z, ν, pbm)
    
    # Hyperplane constraint
    hypln(z, pbm)
    
    # Halfspace constraint
    hlfsp(z, pbm)
        
    # Trust-region penalty constraint
    tr_cnstr(z, η, pbm)
    
    # Virtual buffer penalty constraint
    vb_cnstr(ν, μ, pbm)
    
    # Cost function
    cost_fun(η, μ, z, pbm)
    
    return pbm, z, η, μ
    
end;

## SCP configuration

In [9]:
# Construct problem parameters
params = Dict()
params["solver"] = ECOS
params["opts"]   = Dict("verbose"=>0,"abstol"=>1e-8)
params["scp-verbose"] = true

# Initial reference and constraint data
params["z̄"] = zeros(2);

iter_max = 30
ϵ_conv = 1e-6
params["wtr"] = 10.0
params["wvb"] = 10000.0;

In [10]:
z_hist = [zeros(2) for k = 1:iter_max]
μ_hist = zeros(iter_max)
η_hist = zeros(iter_max)
cost_hist = zeros(iter_max);

## SCP solve step

In [11]:
function solve_pbm!(params, z_hist, η_hist, μ_hist)
    k = 1   
    iter_conv = 0.0
    while true
        pbm, z, η, μ = construct_subpbm(params)                                # Construct subproblem    
        
        exit_flag = solve!(pbm)                                                # Solve subproblem

        z_hist[k], η_hist[k], μ_hist[k] = value(z), value(η)[1], value(μ)[1]   # Save subproblem solution
        cost_hist[k] = z_hist[k][2]

        bool_conv = max(η_hist[k],μ_hist[k]) <= ϵ_conv                         # Check stopping criterion

        if params["scp-verbose"]
            # @printf("Iteration %d | %s | Trust region: %7.1e | Virtual buffer: %7.1e 
            # | Cost: %.2f |\n",k,string(exit_flag),η_hist[k],abs(μ_hist[k]),cost_hist[k])
        end

        if (k == iter_max) || bool_conv                                        # Stop at maximum iterations
            iter_conv = k        
            break
        else
            params["z̄"] .= z_hist[k]                                           # Update reference trajectory
            k += 1
        end
    end
    return iter_conv
end;

In [12]:
iter_conv = solve_pbm!(params, z_hist, η_hist, μ_hist);

<center>
    <br />
    <img src="media/p2_output.png" width="750"/>
    <br />
</center>

## Validating SCP solution via JuMP

In [13]:
function solve_JuMP()
    model = Model(                                       # Define JuMP model and NLP solver
        optimizer_with_attributes(
            Ipopt.Optimizer, "print_level" => 0
        )
    )

    @variable(model, z[1:2])                             # Declare variables

    @NLconstraint(model, z[2] - z[1]^2 == 0)             # Nonconvex quadratic equality constraint

    @constraint(model, Matrix([0.1 1])*z .- 0.6 .== 0)   # Convex constraints
    @constraint(model, Matrix([1 0])*z .- 0.85 .≤ 0)
    @objective(model, Min, z[2])

    set_start_value.(all_variables(model),ones(2))       # Initial guess to NLP solver

    optimize!(model)                                     # Parse and call to solver 
    return JuMP.value.(z), model
end;

In [14]:
z_verify,model = solve_JuMP();


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



### Optimal Solutions

<br><br/>
__Solution using `SCPToolbox.jl`__:

In [15]:
print("\nSCPToolbox.jl: $(z_hist[iter_conv])");


SCPToolbox.jl: [0.7262087350647175, 0.5273791266363734]

<br><br/>
__Verification using `JuMP` and `Ipopt`__:

In [16]:
print("\nJuMP and Ipopt: $(z_verify)");


JuMP and Ipopt: [0.7262087348131022, 0.5273791265186897]

# SCP for Trajectory Optimization

The optimal solution to the simple CLP example presented above is a single point. However, when extending the SCP algorithm to optimal control problems in general, the solution is instead represented as a trajectory. This trajectory solution can be viewed as a sequence of points, each with an associated time, to construct a time history of states. The time history of states will also have a corresponding time history of control inputs. In these problems, the total time horizon may be either fixed as a user-defined quantity, or left free as an additional optimization variable. In the latter case, the original free-final-time non-convex may be linearized about a reference time at each iteration when constructing the local fixed-final-time convex subproblem.

In order to extend the SCP algorithm to trajectory optimization, system dynamics that govern the physical evolution of the vehicle trajectory must be introduced. These dynamics are represented as a differential equation, given by $\dot{x}(t)=f(t, x(t), u(t), p)$, which governs the state derivative $\dot{x}$ as a function of time $t$, the instantaneous state $x$ and control $u$, and possibly some vehicle parameters $p$. The resulting optimal control problem is presented below.

\begin{array}{ll}
\underset{u,\ p}{\text{minimize}} & J(x, u, p) \\
\text{subject to} & \dot{x}(t)=f(t, x(t), u(t), p), \\
& (x(t), p) \in \mathcal{X}(t), \\
& (u(t), p) \in \mathcal{U}(t), \\
& s(t, x(t), u(t), p) \leq 0, \\
& g_{\text {ic}}(x(0), p)=0, \\
& g_{\text {tc}}(x(1), p)=0
\end{array}